In [1]:
# Start by importing H2O and the Deep Learning model you'll need
# (you can find out more about H2ODeepLearningEstimator in H2O's
# Deep Learning Booklet found on their resources page http://h2o.ai/resources/)
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

# initialize an h2o cluster
h2o.init()


H2O cluster uptime:,2 days 20 hours 4 minutes 51 seconds 605 milliseconds
H2O cluster version:,3.7.0.3248
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,1.78 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [2]:
# Download the test and train csvs here: https://www.kaggle.com/c/digit-recognizer/data
# move the test.csv and train.csv files to the same directory as this ipython notebook

# first store the path to your train and test files
train_file = 'train.csv'
test_file = 'test.csv'
# then load your training and testing data into the 'train' and 'test' variables
# using H2O's data frames 
train = h2o.import_file(train_file)
test = h2o.import_file(test_file)

# NOTE: if you didn't move the data files to the same directory as this notebook then you
# need have the entire pathway listed instead of just 'train.csv' 
# (e.g train_file = '/Users/username/Kaggle_Competitions/train.csv')


Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%


In [3]:
# Take a look at the dimensions of your data frames
# notice that test is one column less than train, that's because
# test doesn't contain the response column, listed as 'label', in the train data set
print train.shape
print test.shape

(42000, 785)
(28000, 784)


In [4]:
# If you want to see all the column names you can use train.columns or train.names
# uncomment the below to see examples
# print train.names[0] # looks at the first column name in your data frame
# print test.columns   # gives you all of the column names in your data frame

# if you want to see a brief summary of the data, uncomment and run the following commands
# train.describe()
# test.describe()

In [5]:
# Split your train data set from train.csv into a train and test set (where both sets have a response column)
# you can split the data as you want, the example below splits the data 80/20
# First use .runif to assign a random number to each row, from a uniform set of random numbers between 0 and 1
r = train.runif()   

# all values of r under .8 are assigned to 'train_split' (80 percent)
train_split = train[r  < 0.8]
# all values of r equal or above .8 are assigned 'valid_split' (20 percent)
valid_split = train[r >= 0.8]

In [6]:
# Encode the response column as categorical ('label' in the train.csv data set lists the actual handwritten digit)
train_split["label"] = train_split["label"].asfactor()
valid_split["label"] = valid_split["label"].asfactor()

# set the parameters of your Deep Learning model
model = H2ODeepLearningEstimator(
        distribution="multinomial",
        activation="RectifierWithDropout",
        hidden=[100,200,100],
        input_dropout_ratio=0.2, 
        sparse=True, 
        l1=1e-5, 
        epochs=100)

# train your model on your train_split data set, and then validate it on the valid_split set
# x is the list of column names excluding the response column (all of your features)
# y is the name of the response column ('label')
model.train(
        x= train.names[1:785], 
        y=train.names[0], 
        training_frame=train_split, 
        validation_frame=valid_split)


deeplearning Model Build Progress: [##################################################] 100%


In [7]:
# View the specified parameters of your Deep Learning model
model.params

In [8]:
# Examine the performance of the trained model
model

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1446756221694_10

Status of Neuron Layers: predicting label, 10-class classification, multinomial distribution, CrossEntropy loss, 112,010 weights/biases, 1.5 MB, 3,372,264 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_RMS,momentum,mean_weight,weight_RMS,mean_bias,bias_RMS
,1,706,Input,20.0,,,,,,,,,
,2,100,RectifierDropout,50.0,1e-05,0.0,0.0,0.2,0.0,-0.0,0.1,0.4,0.2
,3,200,RectifierDropout,50.0,1e-05,0.0,0.0,0.0,0.0,-0.0,0.1,0.4,0.2
,4,100,RectifierDropout,50.0,1e-05,0.0,0.0,0.0,0.0,-0.0,0.1,0.2,0.1
,5,10,Softmax,,1e-05,0.0,0.0,0.1,0.0,-1.1,1.1,-8.3,0.6




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.0119909378118
R^2: 0.998563484357
LogLoss: 0.0438406241832

Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
970.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,6 / 976
0.0,1107.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,"10 / 1,117"
0.0,1.0,1004.0,2.0,2.0,0.0,0.0,4.0,5.0,0.0,0.0,"14 / 1,018"
1.0,0.0,4.0,1036.0,0.0,4.0,0.0,2.0,7.0,1.0,0.0,"19 / 1,055"
0.0,1.0,4.0,0.0,936.0,0.0,1.0,1.0,1.0,7.0,0.0,15 / 951
1.0,0.0,1.0,6.0,0.0,855.0,2.0,0.0,1.0,2.0,0.0,13 / 868
4.0,0.0,0.0,0.0,0.0,3.0,971.0,0.0,2.0,0.0,0.0,9 / 980
0.0,2.0,3.0,0.0,0.0,2.0,0.0,1080.0,1.0,2.0,0.0,"10 / 1,090"
1.0,2.0,2.0,4.0,0.0,0.0,2.0,0.0,971.0,1.0,0.0,12 / 983
0.0,2.0,0.0,5.0,5.0,2.0,0.0,5.0,6.0,963.0,0.0,25 / 988



Top-10 Hit Ratios:


...

0,1,2,3,4,5,6,7,8,9,Error,Rate
878.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,6.0,0.0,0.0,11 / 889
0.0,925.0,3.0,1.0,1.0,1.0,0.0,4.0,5.0,0.0,0.0,15 / 940
0.0,2.0,795.0,7.0,4.0,1.0,1.0,2.0,4.0,2.0,0.0,23 / 818
0.0,1.0,7.0,849.0,0.0,4.0,0.0,2.0,4.0,4.0,0.0,22 / 871
3.0,1.0,3.0,0.0,731.0,1.0,6.0,3.0,5.0,14.0,0.0,36 / 767
1.0,1.0,2.0,10.0,0.0,732.0,7.0,0.0,7.0,3.0,0.0,31 / 763
2.0,2.0,2.0,0.0,1.0,7.0,839.0,0.0,1.0,0.0,0.0,15 / 854
3.0,2.0,8.0,1.0,0.0,0.0,0.0,857.0,3.0,13.0,0.0,30 / 887
6.0,3.0,1.0,5.0,1.0,9.0,3.0,1.0,742.0,4.0,0.0,33 / 775
4.0,0.0,0.0,6.0,6.0,4.0,0.0,6.0,4.0,773.0,0.0,30 / 803



Top-10 Hit Ratios:


k,hit_ratio
1,1.0
2,1.0
3,0.9962949
4,0.9974901
5,1.0
6,0.9991633
7,1.0
8,1.0
9,0.9998804
10,0.9999999



Scoring History:


,timestamp,duration,training_speed,epochs,samples,training_MSE,training_r2,training_logloss,training_classification_error,validation_MSE,validation_r2,validation_logloss,validation_classification_error
,2015-11-08 08:48:49,0.000 sec,None,0.0,0.0,nan,nan,nan,nan,nan,nan,nan,nan
,2015-11-08 08:48:54,4.802 sec,15420 rows/sec,1.8,59000.0,0.1,1.0,0.3,0.1,0.1,1.0,0.3,0.1
,2015-11-08 08:49:04,14.779 sec,18095 rows/sec,7.0,236289.0,0.0,1.0,0.2,0.0,0.0,1.0,0.2,0.1
,2015-11-08 08:49:12,23.464 sec,16872 rows/sec,10.5,354583.0,0.0,1.0,0.1,0.0,0.0,1.0,0.2,0.0
,2015-11-08 08:49:22,33.213 sec,17814 rows/sec,15.8,532059.0,0.0,1.0,0.1,0.0,0.0,1.0,0.1,0.0
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2015-11-08 08:51:17,2 min 28.447 sec,20969 rows/sec,84.4,2839786.0,0.0,1.0,0.0,0.0,0.0,1.0,0.1,0.0
,2015-11-08 08:51:27,2 min 37.736 sec,20961 rows/sec,89.7,3017599.0,0.0,1.0,0.0,0.0,0.0,1.0,0.1,0.0
,2015-11-08 08:51:36,2 min 46.625 sec,21007 rows/sec,95.0,3194938.0,0.0,1.0,0.0,0.0,0.0,1.0,0.1,0.0
,2015-11-08 08:51:44,2 min 55.090 sec,21101 rows/sec,100.3,3372264.0,0.0,1.0,0.0,0.0,0.0,1.0,0.1,0.0


In [9]:
# Perform classification on the test set (predict the handwritten digit, given the test set, which is unlabeled)
pred = model.predict(test)

# Take a look at the predictions
pred.head()

predict,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
2,2.58328e-10,4.58576e-10,1,1.26207e-07,1.49441e-11,8.27863e-13,1.34359e-09,2.44748e-07,3.85668e-09,2.22381e-13
0,1,9.87014e-22,4.03992e-14,4.24252e-19,7.84278e-20,3.27868e-11,8.47677e-13,8.37223e-14,8.26366e-16,1.52482e-17
9,2.81387e-06,2.03617e-05,7.30657e-06,0.000835339,0.00377106,0.000872247,1.9292e-06,0.000250928,0.00247517,0.991763
9,0.00084734,0.000156606,0.00233865,0.000845763,0.25564,0.00445571,0.00147996,0.00767056,0.000903631,0.725662
3,2.05177e-06,0.00137416,0.0136499,0.980981,1.2761e-06,0.000648115,3.8037e-07,0.000206657,0.00311694,1.9683e-05
7,3.26777e-07,5.33883e-07,9.99762e-06,2.41914e-05,9.06374e-06,3.05445e-06,1.44662e-10,0.913678,7.37764e-06,0.0862672
0,1,2.05898e-25,1.37185e-15,2.88345e-22,1.45553e-22,9.5252e-16,2.6948e-15,3.49839e-15,4.04579e-18,1.11054e-20
3,1.88687e-17,1.22415e-12,7.31398e-11,1,3.13606e-18,4.6482e-09,1.89086e-19,3.0505e-12,1.07125e-10,4.23183e-12
0,1,5.26201e-24,8.86632e-16,4.76089e-21,1.40486e-21,2.16872e-12,2.78551e-14,8.35894e-15,1.58237e-17,6.80941e-19
3,1.89202e-17,1.6549e-12,4.4385e-11,1,3.89084e-18,1.07972e-08,1.84823e-19,2.17407e-12,1.53148e-10,4.42803e-12


In [10]:
# Grab the prediction column so you can submit it in your Kaggle submission csv
submit_pred = pred[:,0]
submit_pred.head()

predict
2
0
9
9
3
7
0
3
0
3


In [11]:
# Create a dataframe that you can submit to Kaggle with the first column labeled 'ImageID'
# and the second column labeled 'Label'
# Create a H2O dataframe, use a dictionary to create first column, and then cbind the submit_pred column
submission_dataframe = h2o.H2OFrame({'ImageID': list(range(1, len(test)+1))}).cbind(submit_pred)

# Rename the second column (formerely 'Predict') to 'Label'
submission_dataframe.set_name(1,"Label")


Parse Progress: [##################################################] 100%


In [12]:
# Uncomment the line below to take a look at the dataframe
# submission_dataframe

In [13]:
# Export your data into a csv to submit for kaggle
# the first parameter is the dataframe, the second is the path to your submission file
# you can update the name of your submission file to whatever you want
h2o.h2o.export_file(submission_dataframe, path ="submission_h20_3.csv")


Export File Progress: [##################################################] 100%


In [ ]:
# Go to https://www.kaggle.com/c/digit-recognizer/submissions/attach
# and drop in your submission file you should get ~95% accuracy, not bad for a first submission!